In [6]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup

In [3]:
def get_random_headers():
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    return headers

In [ ]:


# URL to the EDGAR filing
url = "https://www.sec.gov/Archives/edgar/data/102909/0001104659-20-018237.txt"

# Send a GET request to the URL
response = requests.get(url, headers=get_random_headers())

# Check if the request was successful
if response.status_code == 200:
    content = response.text
    print(content)  # or process the content as needed
else:
    print("Failed to retrieve the document")


In [20]:
url = "https://www.sec.gov/Archives/edgar/data/1015383/0001493152-20-002565.txt"

# Send a GET request to the URL
response = requests.get(url, headers=get_random_headers())

response.raise_for_status()  # Ensure the request was successful

# Create a BeautifulSoup object
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the starting point
    start_anchor = soup.find('a', attrs={'name': 'gg_002'})
    print(start_anchor)
    # Find the endpoint
    end_anchor = soup.find('a', attrs={'name': 'gg_003'})
    print(end_anchor)

    # Variable to hold all relevant paragraphs
    content = []

    if start_anchor and end_anchor:
        # Start collecting the content from the next sibling of start_anchor
        element = start_anchor.find_next_sibling()
        print(element)

        while element and element != end_anchor:
            print(element)
            if element.name == 'p':
                content.append(element.get_text(strip=True))
            element = element.find_next_sibling()

        # Join the paragraphs with new lines to form the complete section
        full_text = "\n".join(content)
        print(full_text)
    else:
        print("Start or end anchor not found.")

else:
    print("Failed to retrieve the document")

<a name="gg_002"></a>
<a name="gg_003"></a>
<b>ITEM
2. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS.</b>
<b>ITEM
2. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS.</b>

